In [1]:
# Initialize
import numpy as np
import pandas as pd
import re

# Import the processed data

In [2]:
ack1 = pd.read_csv('processed_files\p1_DKOvsCrAT_RO_Kac.csv')
ack2 = pd.read_csv('processed_files\p2_DKOvsSirt3_RO_Kac.csv')

pro1 = pd.read_csv('processed_files\p1_DKOvsCrAT_Protein.csv')
pro2 = pd.read_csv('processed_files\p2_DKOvsSirt3_Protein.csv')

In [3]:
ack1.head()

,Name,Symbol,Gene Symbol,Description,Accession,Entrez,Sequence,Positions in Proteins,Modifications in Proteins,Modifications,...,DKOvsDFC_p_value,CrATKOvsCrATFL_p_value,DKOvsCrATKO_p_value,DFCvsCrATFL_p_value,F,F.p.value,DKOvsDFC_significant,CrATKOvsCrATFL_significant,DKOvsCrATKO_significant,DFCvsCrATFL_significant
0,"Hydroxyacyl-coenzyme A dehydrogenase, mitochon...",Hadh,Hadh,"Hydroxyacyl-coenzyme A dehydrogenase, mitochon...",Q61425,15107.0,TGEGFYKYK,Q61425 [306-314],Q61425 1xAcetyl [K312],1xAcetyl [K7]; 1xTMT6plex [K9]; 1xTMT6plex [N-...,...,1.267950e-09,0.034555,1.355804e-10,0.000235,324.124630,3.071493e-10,1.0,0.0,1.0,0.0
1,"Citrate synthase, mitochondrial",Cs,Cs,"Citrate synthase, mitochondrial OS=Mus musculu...",Q9CZU6,12974.0,IVPNILLEQGKAK,Q9CZU6 [383-395],Q9CZU6 1xAcetyl [K393],1xAcetyl [K11]; 1xTMT6plex [K13]; 1xTMT6plex [...,...,3.025312e-09,0.014144,2.612986e-09,0.297806,199.929464,3.305818e-09,1.0,1.0,1.0,0.0
2,Succinate dehydrogenase [ubiquinone] flavoprot...,Sdha,Sdha,Succinate dehydrogenase [ubiquinone] flavoprot...,Q8K2B3,66945.0,AFGGQSLKFGK,Q8K2B3 [172-182]; A0A1Y7VJ55 [170-180],Q8K2B3 1xAcetyl [K179]; A0A1Y7VJ55 1xAcetyl [K...,1xAcetyl [K8]; 1xTMT6plex [K11]; 1xTMT6plex [N...,...,5.864913e-10,0.096459,4.282730e-10,0.845245,271.052008,7.414188e-10,1.0,0.0,1.0,0.0
3,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,"Trifunctional enzyme subunit alpha, mitochondr...",Q8BMS1,97212.0,FGELALTKESK,Q8BMS1 [327-337],Q8BMS1 1xAcetyl [K334],1xAcetyl [K8]; 1xTMT6plex [K11]; 1xTMT6plex [N...,...,5.235680e-11,0.002444,1.476519e-11,0.000725,533.297696,2.618139e-11,1.0,1.0,1.0,0.0
4,"Citrate synthase, mitochondrial",Cs,Cs,"Citrate synthase, mitochondrial OS=Mus musculu...",Q9CZU6,12974.0,TFKQQHGK,Q9CZU6 [50-57],Q9CZU6 1xAcetyl [K52],1xAcetyl [K3]; 1xTMT6plex [K8]; 1xTMT6plex [N-...,...,9.560681e-10,0.000176,5.296337e-10,0.001231,296.660866,4.752772e-10,1.0,1.0,1.0,0.0


# Making the sausage

This is a hacky bit of code to create a multi-level column header for excel output; it is not very efficient or pretty.

## Acetylpeptides

In [4]:
# make lists of the columns we will want to export in the AcK peptides
ack_protein_info_list = ['Gene Symbol',
                         'Description',
                         'Accession', 
                         'Entrez',]

ack_peptide_info_list = ['Sequence',
                         'Positions in Proteins',
                         'Modifications in Proteins',
                         'Modifications', ]

mitocarta_list = ['MitoCarta2_List',
                  'Matrix',
                  'IMS',]

ack_col_list1 = [ack_protein_info_list, 
                 ack_peptide_info_list, 
                 mitocarta_list]

ack_col_list2 = ack_col_list1.copy()


# go through the RO dataframe and extract the important quantification and limma analysis columns
for i, df in enumerate([ack1, ack2]):
    # the column endings we care about
    for match in ['_log2$', '_ro$', '_Log2FC$', 'p_value$', '_significant$']:
        temp = df.filter(regex=match).columns.tolist()
        if i==0:
            ack_col_list1.append(temp)
        else:
            ack_col_list2.append(temp)
            
                
# generate a group label for the various columns
ack_col_keys = ['Protein', 'Peptide', 'MitoCarta 2.0 Status', 
                'Skeletal Muscle Acetylpeptide Relative Abundance (log2)', 
                'Skeletal Muscle Acetylpeptide Relative Occupancy (log2)']

# account for the 3 components to grab from the limma analysis
ack_col_keys = ack_col_keys + ['limma Analysis on Relative Occupancy']*3

In [5]:
# remove un-wanted columns
ack1 = ack1[np.hstack(ack_col_list1).tolist()]
ack2 = ack2[np.hstack(ack_col_list2).tolist()]

In [6]:
# hack to apply the group label to the dataframe columns for peptides

#### do plex1 
col_stack = pd.DataFrame(data=ack1.columns, columns=['headers'])
col_stack['group'] = np.nan

for cols, key in zip(ack_col_list1, ack_col_keys):
    col_stack.group[col_stack.headers.isin(cols)] = key
    
# prep for and then concat.
col_stack = col_stack.T
col_stack.columns = col_stack.loc['headers']

ack1 = pd.concat([ack1, col_stack])

# generate new columns, aligned to the df, from the dataframe
ack1.columns = pd.MultiIndex.from_arrays(ack1.loc[['group', 'headers']].values)

# get rid of the redundant rows 
ack1.drop(labels=['group', 'headers'], inplace=True)

#### do plex2 
col_stack = pd.DataFrame(data=ack2.columns, columns=['headers'])
col_stack['group'] = np.nan

for cols, key in zip(ack_col_list2, ack_col_keys):
    col_stack.group[col_stack.headers.isin(cols)] = key
    
# prep for and then concat.
col_stack = col_stack.T
col_stack.columns = col_stack.loc['headers']

ack2 = pd.concat([ack2, col_stack])

# generate new columns, aligned to the df, from the dataframe
ack2.columns = pd.MultiIndex.from_arrays(ack2.loc[['group', 'headers']].values)

# get rid of the redundant rows 
ack2.drop(labels=['group', 'headers'], inplace=True)

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
# make the names less cluttered

ra_match = '(?P<geno>.+)_(?P<tmt>.+)_\d_Acetyl_norm_log2$'
ro_match = '(?P<geno>.+)_(?P<tmt>.+)_\d_Acetyl_norm_log2_ro$'
ack_replace = '\g<geno>_\g<tmt>'

for match in [ra_match, ro_match]:
    ack1.rename(columns=lambda x: re.sub(match, ack_replace, x),
                level=1, 
                inplace=True)

    ack2.rename(columns=lambda x: re.sub(match, ack_replace, x),
                level=1, 
                inplace=True)

## Proteins

In [8]:
# make lists of the columns we will want to export in the Proteins
pro_protein_info_list = ['Gene Symbol',
                         'Description',
                         'Accession',
                         'Entrez',
                         'Ensembl',
                         'Chromosome']

pro_id_stats_info_list = ['Coverage [%]',
                          '# Peptides',
                          '# PSMs',
                          '# Protein Unique Peptides',
                          '# Unique Peptides', 
                          'Acetyl (K) Count', 
                          'Acetyl (K) Positions']

pro_col_list1 = [pro_protein_info_list, 
                 mitocarta_list, 
                 pro_id_stats_info_list]

pro_col_list2 = pro_col_list1.copy()


# go through the Protein Dataframe and extract the relevant quantification information
for i, df in enumerate([pro1, pro2]):
    for match in ['_log2$', '_Log2FC$', 'p_value$', '_significant$']:
        temp = df.filter(regex=match).columns.tolist()
    
        if i==0:
            pro_col_list1.append(temp)
        else:
            pro_col_list2.append(temp)

# generate a group label for the various columns
pro_col_keys = ['Protein', 'MitoCarta 2.0 Status', 
                'Identification Statistics',
                'Skeletal Muscle Protein Relative Abundance (log2)']

# account for the 3 components to grab from the limma analysis
pro_col_keys = pro_col_keys + ['limma Analysis on Protein Relative Abundance']*3

In [9]:
pro1 = pro1[np.hstack(pro_col_list1).tolist()].copy()
pro2 = pro2[np.hstack(pro_col_list2).tolist()].copy()

In [10]:
# hack to apply the group label to the dataframe columns for peptides

#### do plex1 
col_stack = pd.DataFrame(data=pro1.columns, columns=['headers'])
col_stack['group'] = np.nan

for cols, key in zip(pro_col_list1, pro_col_keys):
    col_stack.group[col_stack.headers.isin(cols)] = key
    
# prep for and then concat.
col_stack = col_stack.T
col_stack.columns = col_stack.loc['headers']

pro1 = pd.concat([pro1, col_stack])

# generate new columns, aligned to the df, from the dataframe
pro1.columns = pd.MultiIndex.from_arrays(pro1.loc[['group', 'headers']].values)

# get rid of the redundant rows 
pro1.drop(labels=['group', 'headers'], inplace=True)

#### do plex2
col_stack = pd.DataFrame(data=pro2.columns, columns=['headers'])
col_stack['group'] = np.nan

for cols, key in zip(pro_col_list2, pro_col_keys):
    col_stack.group[col_stack.headers.isin(cols)] = key
    
# prep for and then concat.
col_stack = col_stack.T
col_stack.columns = col_stack.loc['headers']

pro2 = pd.concat([pro2, col_stack])

# generate new columns, aligned to the df, from the dataframe
pro2.columns = pd.MultiIndex.from_arrays(pro2.loc[['group', 'headers']].values)

# get rid of the redundant rows 
pro2.drop(labels=['group', 'headers'], inplace=True)

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
# make the names less cluttered

ra_match = '(?P<geno>.+)_(?P<tmt>.+)_\d_Proteins_norm_log2$'
pro_replace = '\g<geno>_\g<tmt>'

pro1.rename(columns=lambda x: re.sub(ra_match, pro_replace, x),
            level=1, 
            inplace=True)

pro2.rename(columns=lambda x: re.sub(ra_match, pro_replace, x),
            level=1, 
            inplace=True)

# Export

In [13]:
# export to excel 
writer = pd.ExcelWriter('processed_files/SKM DKO 2x10plex_RO_limma.xlsx', 
                        options={'strings_to_numbers': True}, 
                        engine='xlsxwriter')

ack1.to_excel(writer, sheet_name='Plex1 Acetylpeptide SUMMARY')
ack2.to_excel(writer, sheet_name='Plex2 Acetylpeptide SUMMARY')

pro1.to_excel(writer, sheet_name='Plex1 Protein SUMMARY')
pro2.to_excel(writer, sheet_name='Plex2 Protein SUMMARY')

# to include the proteome discoverer 2.2 exported data,
# uncomment the following code:

# peptides_file = 'pd22_exports/pd22_export_Williams_2019_PeptideGroups.txt'
# proteins_file = 'pd22_exports/pd22_export_Williams_2019_Proteins.txt'
# pep_pd22 = pd.read_csv(peptides_file, sep='\t', low_memory=False)
# pro_pd22 = pd.read_csv(proteins_file, sep='\t', low_memory=False)
# pep_pd22.to_excel(writer, sheet_name='Peptide Groups RAW', index=False)
# pro_pd22.to_excel(writer, sheet_name='Proteins RAW', index=False)

writer.save()